<a href="https://colab.research.google.com/github/raj-sahu/SignToText/blob/master/signHack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DRIVE **SETUP** 


In [2]:
from google.colab import drive # import drive from google colab
ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)
drive.mount(ROOT) 

/content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd '/content/drive/My Drive/HackSign'
!ls

/content/drive/My Drive/HackSign
all		     lsa64_preprocessed.zip  __pycache__
lsa64_hand_videos    lsa64_raw.zip	     Sign4Good
lsa64_positions.mat  mediapipeModels	     SignToText4.h5


In [ ]:
!wget https://drive.google.com/file/d/1yhfPpI2iJzPXyx4C7MYR6IPZC3YuuYaL/view?usp=sharing 

In [ ]:
!unzip lsa64_preprocessed.zip

# IMPORTS

In [ ]:
!wget https://github.com/google/mediapipe/blob/master/mediapipe/models/palm_detection.tflite
!wget https://github.com/google/mediapipe/tree/master/mediapipe/models/hand_landmark.tflite

--2020-07-31 22:43:48--  https://github.com/google/mediapipe/blob/master/mediapipe/models/palm_detection.tflite
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘palm_detection.tflite’

palm_detection.tfli     [ <=>                ]  78.53K  --.-KB/s    in 0.03s   

2020-07-31 22:43:48 (2.90 MB/s) - ‘palm_detection.tflite’ saved [80416]

--2020-07-31 22:43:55--  https://github.com/google/mediapipe/tree/master/mediapipe/models/hand_landmark.tflite
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/google/mediapipe/blob/master/mediapipe/models/hand_landmark.tflite [following]
--2020-07-31 22:43:55--  https://github.com/google/mediapipe/blob/master/mediapipe/models/hand_lan

In [ ]:
!mkdir mediapipeModels
!mv palm_detection.tflite ./mediapipeModels/palm_detection.tflite
!mv hand_landmark.tflite  ./mediapipeModels/hand_landmark.tflite

In [ ]:
%ls

In [ ]:
!ls lsa64_hand_videos/ -X 

In [ ]:
!wget https://doc-0o-5g-docs.googleusercontent.com/docs/securesc/9f0456s7r9fdcnmu7t0tniodjolt87v0/1occffc7f59b7o7pc5ci3168frbdnhkm/1596241425000/06778836722188637978/18337363771433511205/1C7k_m2m4n5VzI4lljMoezc-uowDEgIUh?e=download&authuser=0&nonce=0323pmr0vpjqc&user=18337363771433511205&hash=d2g6bk3kt40fnv20sf1gpn71m2t9uc7l

In [ ]:
!unzip lsa64_raw.zip

In [ ]:
!ls all -x

In [ ]:
signtoindex={ 
    "Accept": 49,
    "Appear": 52,
    "Argentina": 23,
    "Away": 12,
    "Barbecue": 44,
    "Bathe": 57,
    "Birthday": 29,
    "Bitter": 18,
    "Born": 14,
    "Breakfast": 30,
    "Bright": 4,
    "Buy": 58,
    "Call": 16,
    "Candy": 45,
    "Catch": 54,
    "Chewing-gum": 46,
    "Coin": 34,
    "Colors": 6,
    "Copy": 59,
    "Country": 25,
    "Dance": 56,
    "Deaf": 41,
    "Drawer": 13,
    "Enemy": 9,
    "Find": 63,
    "Food": 22,
    "Give": 62,
    "Green": 2,
    "Help": 55,
    "Hungry": 32,
    "Last name": 26,
    "Learn": 15,
    "Light-blue": 5,
    "Man": 11,
    "Map": 33,
    "Milk": 20,
    "Mock": 28,
    "Music": 35,
    "Name": 38,
    "None": 37,
    "Opaque": 0,
    "Patience": 39,
    "Perfume": 40,
    "Photo": 31,
    "Realize": 61,
    "Red": 1,
    "Red2": 7,
    "Rice": 43,
    "Run": 60,
    "Ship": 36,
    "Shut down": 51,
    "Skimmer": 17,
    "Son": 10,
    "Spaghetti": 47,
    "Sweet milk": 19,
    "Thanks": 50,
    "To land": 53,
    "Trap": 42,
    "Uruguay": 24,
    "Water": 21,
    "Where": 27,
    "Women": 8,
    "Yellow": 3,
    "Yogurt": 48}

In [ ]:
indextosign={idx:sign for (sign,idx) in signtoindex.items()}

# DATA SPLIT

In [5]:
import os
import shutil
import glob

In [ ]:
os.chdir('all')
videos=os.listdir(os.getcwd())
for i in range(64):
  if os.path.isdir(indextosign[i]) is False:
     os.makedirs(indextosign[i])
  p=str(i)
  if(i<10):
    p='0'+p
  for file in glob.glob('0'+p+'_*'):
    shutil.move(file,indextosign[i])
for file in glob.glob('064_*'):
  os.remove(file)
os.chdir('../')
%ls


In [6]:
os.getcwd()

'/content/drive/My Drive/HackSign'

## MODEL

In [7]:
import numpy as np
import warnings
import cv2
import os
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, MaxPooling2D, Reshape, LSTM
from keras.models import Model, Sequential, load_model
from sklearn.preprocessing import LabelBinarizer
warnings.filterwarnings("ignore", category=FutureWarning)

Using TensorFlow backend.


In [8]:
def vid2frames(vid_dir, frame_limit,noofclass):
    """
    Isolate hands and from video and split video into individual frames

        PARAMETERS:
        vid_dir (str): Directory of videos

        frame_limit (int): Maximum number of frames per video

        RETURN:
        X_train: Training Frames

        Y_train: Labels of each frame

    """

    data = []

    labels = os.listdir(vid_dir)[0:noofclass]
    print(labels)

    for label in labels:
        print("Processing ", label)
        videos = os.listdir(vid_dir+'/'+label)
        for vid in videos:
            i = 0
            cap = cv2.VideoCapture(vid_dir+'/'+label+'/'+vid)
            while(cap.isOpened() and i < frame_limit):

                ret, frame = cap.read()

                if ret == False:
                    break

                image = frame
                result = image.copy()
                image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

                lower = np.array([41, 156, 49])
                upper = np.array([179, 255, 255])

                mask = cv2.inRange(image, lower, upper)
                result = cv2.bitwise_and(result, result, mask=mask)

                result = cv2.resize(result, (256, 256))

                data.append([result, label])

                i += 1

    X_train = []
    Y_train = []

    for result, label in data:
        X_train.append(result)
        Y_train.append(label)

    return(X_train, Y_train)

In [ ]:
No_Class=4
X_train, Y_train = vid2frames('./all', 100,No_Class)

['Opaque', 'Red', 'Green', 'Yellow']
Processing  Opaque
Processing  Red
Processing  Green
Processing  Yellow


In [ ]:
encoder = LabelBinarizer()
Y_encoded_train =  encoder.fit_transform(Y_train)
X_train = np.array(X_train)
len(Y_train),Y_encoded_train.shape,X_train.shape

(14748, (14748, 3), (14748, 256, 256, 3))

In [ ]:
model = Sequential()
model.add(Conv2D(32 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu' , input_shape = (256,256,3)))
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(128 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(256 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Reshape((16, 16*256)))
model.add(LSTM(64, return_sequences=True, input_shape=(1,256)))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(32))
model.add(Dense(No_Class-1, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 256, 256, 32)      896       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 128, 128, 64)      18496     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 64, 64, 128)       73856     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 32, 32, 256)      

In [ ]:
model.fit(X_train, Y_encoded_train, epochs=16, batch_size=32)

Epoch 1/16
14748/14748 [==============================] - 76s 5ms/step - loss: 0.0885 - accuracy: 0.9745
Epoch 2/16
 9152/14748 [=================>............] - ETA: 24s - loss: 0.0317 - accuracy: 0.9903

KeyboardInterrupt: ignored

In [ ]:
 model.save("SignToText"+str(No_Class)+".h5")

In [9]:
import numpy as np
import cv2
boundaries = [
    ([0, 120, 0], [140, 255, 100]),
    ([25, 0, 75], [180, 38, 255])
]


def handsegment(frame):
    lower, upper = boundaries[0]
    lower = np.array(lower, dtype="uint8")
    upper = np.array(upper, dtype="uint8")
    mask1 = cv2.inRange(frame, lower, upper)

    lower, upper = boundaries[1]
    lower = np.array(lower, dtype="uint8")
    upper = np.array(upper, dtype="uint8")
    mask2 = cv2.inRange(frame, lower, upper)
    mask = cv2.bitwise_or(mask1, mask2)
    output = cv2.bitwise_and(frame, frame, mask=mask)
    cv2.imshow("images", mask)
    cv2.imshow("images", output)
    return output

In [10]:
frame = cv2.imread("test.png")
handsegment(frame)

error: ignored